In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

boston_housing = tf.keras.datasets.boston_housing
models = tf.keras.models
layers = tf.keras.layers

In [2]:
# load data
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()
print('train_data.shape = ' + str(train_data.shape))
print('test_data.shape = ' + str(test_data.shape))
print('train_targets = ['  + ', '.join([str(t) for t in train_targets[:3]]) + ', ..., ' 
      + ', '.join([str(t) for t in train_targets[-3:]]) + ']')

train_data.shape = (404, 13)
test_data.shape = (102, 13)
train_targets = [15.2, 42.3, 50.0, ..., 19.4, 19.4, 29.1]


In [3]:
# normalize the features (subtract mean and divide by standard deviation)
mean = train_data.mean(axis=0)
std = train_data.std(axis=0)
print('mean.shape = ' + str(mean.shape))
print('std.shape = ' + str(std.shape))

train_data -= mean
train_data /= std
test_data -= mean
test_data /= std

mean.shape = (13,)
std.shape = (13,)


In [4]:
# model definition
def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape = (train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

In [5]:
# build and train model on one fold of training/validation split
def build_and_train_model( fold, num_folds = 4, num_epochs = 100 ):
    num_val_samples = len(train_data) // num_folds
    start_idx = fold*num_val_samples
    end_idx = (fold+1)*num_val_samples
    val_data = train_data[start_idx:end_idx]
    val_targets = train_targets[start_idx:end_idx]
    partial_train_data = np.concatenate(
        [train_data[:end_idx], train_data[end_idx:]], axis = 0)
    partial_train_targets = np.concatenate(
        [train_targets[:end_idx], train_targets[end_idx:]], axis = 0)
    model = build_model()
    history = model.fit(partial_train_data, partial_train_targets, epochs=num_epochs, 
                        batch_size=1, verbose=0)
    return (model, history)

In [6]:
from concurrent.futures import ProcessPoolExecutor

k = 4
num_epochs = 10
all_scores = []
futures = []
with ProcessPoolExecutor() as executor:
    for i in range(k):
        future = executor.submit(build_and_train_model, i, 
                                 num_folds = k, num_epochs = num_epochs )
        futures.append(future)
    models = []
    histories = []
    for future in futures:
        model, history = future.result() 
        models.append(models)
        histories.append(history)

W0903 12:27:02.676187 4515636672 deprecation.py:506] From //anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0903 12:27:02.677651 4515636672 deprecation.py:506] From //anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0903 12:27:02.676197 4515636672 deprecation.py:506] From //anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.py

TypeError: can't pickle _thread.RLock objects